In [1]:
#Notebook, BeautifulSoup, Pandas, and Requests/Splinter
from bs4 import BeautifulSoup as bs
import pandas as pd
import pymongo
import requests
from splinter import Browser

In [3]:
cd_fp = "C:/Users/norma/Anaconda3/chromedriver.exe"
browser = Browser('chrome', cd_fp, headless=True)

In [10]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)
db = client.MarsDB

In [ ]:
db2 = client.classDB
classroom = db2.classroom.find()

# Iterate through each student in the collection
for student in classroom:
    print(student)
    
print('\n')
    
for student in db2.mw.find():
    print(student)

## NASA Mars News

In [ ]:
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

In [ ]:
html = browser.html
soup = bs(html, 'html.parser')
first_article = soup.find('li', class_ = 'slide')
print(first_article)
nasa_title = first_article.find('h3').text.strip()
print(nasa_title)
nasa_abstract = first_article.find('div', class_ = 'article_teaser_body').text.strip()
print(nasa_abstract)

In [ ]:
result = db.News.drop()
result = db.News.insert_one({'title':nasa_title,'abstract':nasa_abstract})

## JPL Mars Space Images - Featured Image

In [4]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [ ]:
# soup = bs(browser.html, 'html.parser')
# body = soup.find('body')
# with open('jpl_body.html','w') as f:
#     f.write(body.prettify())
    
browser.links.find_by_partial_text('FULL IMAGE').click()

# soup2 = bs(browser.html, 'html.parser')
# body = soup2.find('body')
# with open('jpl_full_image.html','w') as f:
#     f.write(body.prettify())
    
browser.links.find_by_partial_text('more info').click()

soup3 = bs(browser.html, 'html.parser')
# body = soup3.find('body')
# with open('jpl_more_info.html','w') as f:
#     f.write(body.prettify())
    
featured_image_url = 'https://www.jpl.nasa.gov' + soup3.find('figure').find('a')['href'])

In [ ]:
result = db.Image.drop()
result = db.Image.insert_one({'url':featured_image_url})

## Mars Weather

In [ ]:
#twitter.com/marswxreport?lang=en
url = 'http://twitter.com/marswxreport?lang=en'
#browser.visit(url)
resp = requests.get(url)

In [ ]:
soup = bs(resp.content, 'html.parser')
mars_weather=soup.find('p', class_='TweetTextSize').text.strip()
print(mars_weather)

In [ ]:
result = db.Weather.drop()
result = db.Weather.insert_one({'conditions':mars_weather})

## Mars Facts

In [ ]:
url = 'https://space-facts.com/mars/'
dfs = pd.read_html(url)
df_mars_facts = dfs[0]
df_mars_facts.columns = ['Attribute','Value']
df_mars_facts

In [ ]:
df_mars_facts.to_html()

In [ ]:
result = db.Facts.drop()
result = db.Facts.insert_one({'table_html':df_mars_facts.to_html()})

## Mars Hemispheres

In [5]:
url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(url)

In [6]:
soup = bs(browser.html,'html.parser')
# body = soup.find('body')
# with open('hemispheres.html','w') as f:
#     f.write(body.prettify())


In [7]:
# Refinement of Jeab's code
# divs = soup.find_all('div', class_='item')
# for div in divs:
#     key_word = div.find('h3').text
#     browser.find_link_by_partial_text(key_word).click()
#     img_url = browser.find_link_by_text('Sample').first['href']
#     print(img_url)
#     browser.back()

In [8]:
hemisphere_image_urls = list()
divs = soup.find_all('div', class_='description')
for div in divs:
    key_word = div.find('h3').text
#    print(key_word)
    browser.links.find_by_partial_text(key_word).click()
    
    # Now our browser is on the linked page
    soup2 = bs(browser.html,'html.parser')
#     body = soup.find('body')
#     with open(f'{key_word}.html','w') as f:
#         f.write(body.prettify())
    
    downloads = soup2.find('div', class_='downloads')
    img_url = downloads.find('a')['href']
    
    hemisphere_image_urls.append({'title':key_word.replace(' Enhanced',''),'img_url':img_url})

    # Go back to the original page
    browser.back()
    
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [11]:
result = db.Hemispheres.drop()
result = db.Hemispheres.insert_one({'title_and_image_list':hemisphere_image_urls})

## MarsDB

In [ ]:
x = db.News.find()
print(type(x))
print(x.next)
print(x[0])

# for y in x:
#     print(y)

In [ ]:
mars_info = {'News' : db.News.find_one({}, {'_id':False}),
             'Image' : db.Image.find_one({}, {'_id':False}),
             'Weather' : db.Weather.find_one({}, {'_id':False}),
             'Facts' : db.Facts.find_one({}, {'_id':False}),
             'Hemispheres' : db.Hemispheres.find_one({}, {'_id':False})
            }
mars_info